# 2018/2019 - Task List 1

1. Generate random variable from prefered dystribution using Pyro (pyro.sample)
    
    - animate how distribution of values changes
    - animate histograms of values
    - start with empty list of values, generate new samples, generate histogram
    
    
2. Write a simulator (and exact solution utilizing Bayes theorem) for chances to be ill on a certain disease. We know that it affects from about 1 to 100 out of 50,000 people. There was developed a test to check whether the person has the disease and it is quite accurate: the probability that the test result is positive (suggesting the person has the disease), given that the person does not have the disease, is only 2 percent; the probability that the test result is negative (suggesting the person does not have the disease), given that the person has the disease, is only 1 percent. When a random person gets tested for the disease and the result comes back positive, what is the probability that the person has the disease? Check whole parameter space and visualise results.

    - it must be a simulator!
    - sample from distribution using given probabilities
    - repeate experiment and compare with Bayes equation


## Required imports

In [5]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import plotly.graph_objs as go
import pyro
import torch

from matplotlib import animation, rc
from IPython.display import HTML
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot


init_notebook_mode(connected=True)

## Generating single value from normal distribution with given mean and variance

In [ ]:
mean = 0
variance = 10
x = pyro.sample("normal_sampling", pyro.distributions.Normal(mean, variance))
print(x)

## Sampling from categorical probabilities with given probabilities

In [ ]:
x = pyro.sample("categorical_sampling", pyro.distributions.Categorical(logits = torch.tensor([0.5, 0.5])))
print(x)

## Animating data

In [ ]:
def animate():
    # First set up the figure, the axis, and the plot element we want to animate
    fig, ax = plt.subplots()

    ax.set_xlim((-2, 2))
    ax.set_ylim((0, 4))

    line, = ax.plot([], [], lw=2)
    
    # animation function. This is called sequentially
    def sin(variance):
        mean = 0
        x = [pyro.sample("normal_sampling", pyro.distributions.Normal(mean, variance)) for _ in range(100)]
        line = plt.hist(x, density=True, bins=30)
        return (line,)

    # initialization function: plot the background of each frame
    def init():
        line.set_data([], [])
        return (line,)
    # call the animator. blit=True means only re-draw the parts that have changed.
    anim = animation.FuncAnimation(fig, sin, init_func=init,
                                   frames=100, interval=20, blit=True)
    return HTML(anim.to_jshtml())

animate()


## Drawing histogram

In [ ]:
x = np.random.normal(size = 1000)
plt.hist(x, density=True, bins=30)
plt.ylabel('Probability');

## Task 1

In [ ]:
## Insert solution here

mean = 0
variances_count = 10

def animate():
    def update_hist(variance, mean):
        x = [pyro.sample("normal_sampling", pyro.distributions.Normal(mean, variance)) for _ in range(500)]
        plt.cla()
        plt.hist(x, bins=30)

    fig = plt.figure()
    hist = plt.hist([])

    anim = animation.FuncAnimation(fig, update_hist, variances_count, fargs=(mean,))
    return HTML(anim.to_jshtml())

animate()

## Task 2

Write a simulator (and exact solution utilizing Bayes theorem) for chances to be ill on a certain disease. We know that it affects from about 1 to 100 out of 50,000 people. There was developed a test to check whether the person has the disease and it is quite accurate: the probability that the test result is positive (suggesting the person has the disease), given that the person does not have the disease, is only 2 percent; the probability that the test result is negative (suggesting the person does not have the disease), given that the person has the disease, is only 1 percent. When a random person gets tested for the disease and the result comes back positive, what is the probability that the person has the disease? Check whole parameter space and visualise results.

    - it must be a simulator!
    - sample from distribution using given probabilities
    - repeate experiment and compare with Bayes equation


In [29]:
## Insert solution here

population_size = 50000
disease_range = range(1, 101)

negative_for_ill_probability = 0.01
positive_for_ill_probability = 1 - negative_for_ill_probability
positive_for_health_probability = 0.02
negative_for_health_probability = 1 - positive_for_health_probability


def naive_bayes(disease_count):
    disease_probability = disease_count/population_size
    health_probability = 1 - disease_probability
    positive_if_ill_probability = disease_probability*positive_for_ill_probability
    is_ill_probability = positive_if_ill_probability/(positive_if_ill_probability
                                                      + health_probability*positive_for_health_probability)
    return is_ill_probability


def theorem_probabilities():
    probabilities_results = {}
    for disease_count in disease_range:
        theorem_probability = naive_bayes(disease_count)
        probabilities_results.update({disease_count:theorem_probability})
    return probabilities_results


class BayesExperiment:
    
    def __init__(self, disease_count):
        self.disease_count = disease_count
        
        self.ill_positives = 0
        self.ill_negatives = 0
        self.health_positives = 0
        self.health_negatives = 0

    @property
    def positives(self):
        return self.ill_positives + self.health_positives

    def _generate_population(self):
        disease_probability = self.disease_count/population_size
        health_probability = 1 - disease_probability
        distribution = pyro.distributions.Categorical(probs=torch.tensor([health_probability, 
                                                                           disease_probability]))
        population = [pyro.sample("population_distribution", distribution) 
                      for _ in range(population_size)]
        return population

    def bayes_experiment(self):
        population = self._generate_population()
        for person_state in population:
            self.perform_test(person_state)

    def perform_test(self, person_state):
        if person_state == 0:
            self._perform_test(person_state, negative_for_health_probability, 
                               positive_for_health_probability, distribution_name="health test distribution")
        else:
            self._perform_test(person_state, negative_for_ill_probability, 
                               positive_for_ill_probability, distribution_name="ill test distribution")

    def _perform_test(self, person_state, negative_probability, positive_probability, distribution_name="default"):
        test_result = pyro.sample(distribution_name, 
                                  pyro.distributions.Categorical(probs=torch.tensor([negative_probability, 
                                                                                      positive_probability])))
        self._update_results(person_state, test_result)

    def _update_results(self, person_state, test_result):
        if person_state == test_result:
            if person_state:
                self.ill_positives += 1
            else:
                self.health_negatives += 1
        else:
            if person_state:
                self.ill_negatives += 1
            else:
                self.health_positives += 1

    def plot_data(self):
        trace0 = go.Bar(
            x=['Ill positives', 'Health positives', 'Ill negatives', 'Health negatives'],
            y=[self.ill_positives, self.health_positives, self.ill_negatives, self.health_negatives],
            name='Bayes experiment',
            marker=dict(
                color='rgb(49,130,189)'
            )
        )

        data = [trace0]
        layout = go.Layout(
            xaxis=dict(tickangle=-45),
            barmode='group',
        )

        fig = go.Figure(data=data, layout=layout)
        iplot(fig, filename='angled-text-bar')



In [31]:
disease_count = 1000

experiment = BayesExperiment(disease_count)
experiment.bayes_experiment()
print(f"{experiment.ill_positives} \n"
      f"{experiment.ill_negatives} \n"
      f"{experiment.health_positives} \n"
      f"{experiment.health_negatives} \n")
print(f"{experiment.ill_positives/experiment.positives}")
print(naive_bayes(disease_count))
experiment.plot_data()

936 
14 
965 
48085 

0.49237243556023147
0.5025380710659898
